In [182]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [183]:
cd ~/demres

/Users/zurfarosa/demres


In [184]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
from datetime import date, timedelta

import demres
from demres.common.constants import entry_type
from demres.common import codelists,druglists
from demres.common.process_pt_features import *
from demres.common.process_entries import *
from demres.demins.constants import Study_Design as sd
from demres.demins.statistical_functions import *
from common.helper_functions import *

In [185]:
pd.set_option('display.max_columns', None)

## Process raw CSV files

In [5]:
# create_pegmed()

In [6]:
# create_pegprod()

In [7]:
# create_prescriptions()

In [8]:
# create_consultations()

In [9]:
# create_clinicals()

In [10]:
# create_tests()

In [11]:
# create_referrals()

In [12]:
# create_immunisations()

In [13]:
# create_medcoded_entries()

## Create basic pt_features dataframe
*pt_features will contain all the variables (e.g. age, male gender, insomnia count) used in the logistic regression*

In [14]:
# specify subtype of dementia we're interested in - 'vascular','alzheimers' or 'all_dementia'
subtype = 'all_dementia' 

In [15]:
# all_entries = get_all_entries()

In [16]:
# pt_features = create_pt_features()

In [17]:
# pt_features = get_index_date_and_caseness_and_add_final_dementia_subtype(all_entries,pt_features)

In [18]:
# pt_features = only_include_specific_dementia_subtype(pt_features,subtype=subtype)

In [19]:
# pt_features = add_data_start_and_end_dates(all_entries,pt_features)

In [20]:
# len(pt_features[pt_features['isCase']==True]),len(pt_features[pt_features['isCase']==False])

In [21]:
# pt_features.to_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'.csv',index=False)

In [22]:
# for window in sd.exposure_windows:
#     print(window['name'],' being matched')    
#     pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)
#     pt_features = match_cases_and_controls(pt_features,req_yrs_post_index=sd.req_yrs_post_index,start_year=abs(window['start_year']))
#     pt_features = delete_unmatched_cases_and_controls(pt_features)
#     pt_features.to_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+ window['name'] +'.csv',index=False)

## Add derived variables to pt_features 
*e.g. insomnia count, presence of diabetes, consultation count*

In [23]:
medcoded_entries = pd.read_hdf('hdf/medcoded_entries.hdf')

In [24]:
prescriptions = pd.read_hdf('hdf/prescriptions.hdf')

In [73]:
pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+sd.exposure_windows[0]['name']+'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)
prescs = pd.merge(prescriptions,pt_features[['patid','index_date']],how='left',on='patid')
prescs = prescs[pd.notnull(prescs['qty'])] #remove the relatively small number of prescriptions where the quantity is NaN

In [74]:
pegprod = pd.read_csv('data/dicts/proc_pegasus_prod.csv')
prescs = pd.merge(prescs,pegprod[['prodcode','substance strength','route','drug substance name']],how='left')

In [75]:
prescs.sample(5)

,patid,eventdate,sysdate,consid,prodcode,staffid,textid,bnfcode,qty,ndd,numdays,numpacks,packtype,issueseq,index_date,substance strength,route,drug substance name
8632474,12878093,1997-02-05,1997-02-05,1873168,12,6093,2903,64,500.0,30.0,0,0.0,6,1,NaT,3.35g/5ml,Oral,Lactulose
3036986,26277028,2013-01-28,2013-01-21,1769699,38928,9028,13,171,14.0,2.0,0,0.0,921,2,NaT,1.25gram + 400unit,Oral,Calcium carbonate/Colecalciferol
23074725,6339236,2007-04-24,2007-04-20,815378,802,4236,5,86,7.0,1.0,0,0.0,1,1,2006-08-16,40mg,Oral,Simvastatin
5110708,3228054,2007-09-17,2007-09-17,770046,23,4054,102213,85,14.0,0.0,0,0.0,1,1,NaT,500mg,Oral,Metformin hydrochloride
6537726,1806074,2005-07-20,2005-07-20,174286,2,6074,8,17,28.0,1.0,0,0.0,323,1,NaT,2.5mg,Oral,Bendroflumethiazide


In [91]:
all_drugs = [drug for druglist in druglists.all_druglists for drug in druglist['drugs'] ]

In [77]:
# relev_prescs = prescs[prescs['drug substance name'].isin(all_drugs)]

In [92]:
prodcodes = get_prodcodes_from_drug_name(all_drugs)
relev_prescs = prescs.loc[prescs['prodcode'].isin([prodcodes])]

In [94]:
relev_prescs.sample(5)

,patid,eventdate,sysdate,consid,prodcode,staffid,textid,bnfcode,qty,ndd,numdays,numpacks,packtype,issueseq,index_date,substance strength,route,drug substance name
10709674,9050117,2005-02-24,2005-02-24,1095421,49,63117,36,47,30.0,1.0,0,0.0,1,3,2009-10-25,25mg,Oral,Amitriptyline hydrochloride
2111836,1887021,2009-09-30,2009-09-30,314578,476,14021,18,145,28.0,1.0,0,0.0,9763,1,2007-08-08,10mg,Oral,Citalopram hydrobromide
9607880,9815106,2009-04-23,2009-04-23,1682290,6795,4106,36,221,28.0,1.0,0,0.0,1,0,NaT,15mg,Oral,Mirtazapine
27558870,17069284,2009-01-14,2009-01-14,2984406,84,8284,118,47,28.0,1.0,0,0.0,2,1,NaT,25mg,Oral,Dosulepin hydrochloride
27615496,27890284,2004-09-27,2004-09-27,4107251,49,9284,2372,47,28.0,1.0,0,0.0,1,3,NaT,25mg,Oral,Amitriptyline hydrochloride


In [95]:
# Create new columns ('amount' and 'unit', extracted from the 'substrance strength' string)
amount_and_unit = relev_prescs['substance strength'].str.extract('([\d\.]+)([\d\.\+ \w\/]*)',expand=True)
amount_and_unit.columns=['amount','unit']
amount_and_unit.amount = amount_and_unit.amount.astype('float')
reformatted_prescs = pd.concat([relev_prescs,amount_and_unit],axis=1).drop(['numpacks','numdays','packtype','issueseq'],axis=1)

# Convert micrograms to mg
micro_mask = reformatted_prescs['unit'].str.contains('microgram',na=False,case=False)
reformatted_prescs.loc[micro_mask,'amount'] /= 1000
reformatted_prescs.loc[micro_mask,'unit'] = 'mg'

#Convert mg/Xml to mg for simplicity
micro_mask = reformatted_prescs['unit'].str.contains('mg/',na=False,case=False)
reformatted_prescs.loc[micro_mask,'unit'] = 'mg'

# Create a 'total_amount' column - used to calculate each pt's PDDs for a given drug.
reformatted_prescs['total_amount'] = reformatted_prescs['qty']*reformatted_prescs['amount']

#Change all 'numeric daily doses' (NDD) from 0 (this appears to be the default in the CPRD data) to 1.
#Note that an NDD of 2 means 'twice daily'
reformatted_prescs.loc[reformatted_prescs['ndd'] == 0,'ndd']=1

#Only use prescriptions belonging to the main exposure window (not the ones used in sensitivity analysis)
start_year = timedelta(days=(365*abs(sd.exposure_windows[1]['start_year'])))
end_year = timedelta(days=(365*abs(sd.exposure_windows[1]['start_year']+sd.window_length_in_years)))
timely_presc_mask = (reformatted_prescs['eventdate']>=(reformatted_prescs['index_date']-start_year)) & (reformatted_prescs['eventdate']<=(reformatted_prescs['index_date']-end_year))
timely_prescs = reformatted_prescs[timely_presc_mask]


In [99]:
timely_prescs.sample(15)

,patid,eventdate,sysdate,consid,prodcode,staffid,textid,bnfcode,qty,ndd,index_date,substance strength,route,drug substance name,amount,unit,total_amount
31366534,13431323,1999-05-25,1999-05-25,2173604,4329,53323,34,47,30.0,1.0,2006-02-10,20mg,Oral,Mianserin hydrochloride,20.0,mg,600.0
30784445,8250320,1999-05-14,2000-06-28,2213044,500,12320,330,96,20.0,3.0,2008-07-31,3mg,Oromucosal buccal,Prochlorperazine maleate,3.0,mg,60.0
4506422,1784051,2003-01-28,2003-01-28,313694,35,9051,36,56,10.0,1.0,2008-04-07,5mg,Oral,Nitrazepam,5.0,mg,50.0
22940840,6954234,2002-06-18,2002-06-18,888813,83,5234,3661,47,100.0,2.5,2008-10-17,10mg,Oral,Amitriptyline hydrochloride,10.0,mg,1000.0
27426067,6945284,1999-07-13,1999-07-13,1200735,432,9284,16,218,28.0,1.0,2006-07-11,100mg,Oral,Carbamazepine,100.0,mg,2800.0
14437243,9304154,2001-05-08,2001-05-08,1098896,4020,84154,43,47,56.0,2.0,2008-06-25,150mg,Oral,Trazodone hydrochloride,150.0,mg,8400.0
30071468,631315,2002-06-25,2002-06-25,177918,3105,4315,52,56,100.0,2.0,2007-11-28,30mg,Oral,Flurazepam hydrochloride,30.0,mg,3000.0
11364316,1935124,2000-10-06,2000-10-06,295978,114,8124,15252,47,28.0,1.0,2010-02-10,70mg,Oral,Lofepramine hydrochloride,70.0,mg,1960.0
29831450,1570311,2003-05-08,2003-05-08,330883,3355,6311,14,47,28.0,1.0,2009-11-16,50mg,Oral,Trazodone hydrochloride,50.0,mg,1400.0
10322525,8165114,2002-05-23,2002-05-23,1523718,85,4114,35,96,84.0,1.0,2008-12-11,5mg,Oral,Prochlorperazine maleate,5.0,mg,420.0


In [100]:
timely_prescs.to_hdf('hdf/timely_prescs.hdf','timely_prescs',mode='w',format='table')

In [134]:
pdds = pd.DataFrame(columns=['drug_name','drug_type','pdd'])

In [211]:
for druglist in druglists.all_druglists:
    prodcodes = get_prodcodes_from_drug_name(druglist['drugs'])
    druglist_prescs = timely_prescs.loc[timely_prescs['prodcode'].isin(prodcodes)]

    # Remove prescriptions if they are not of the route (e.g. oral) specified on the druglist
    druglist_prescs = druglist_prescs.loc[prescs['route'].str.contains(druglist['route'],na=False,case=False)]


    #Calculate the prescribed daily dose (PDD) for each drug (e.g. 'citalopram') in the druglist (e.g. 'antidepressants')
    for drug in druglist['drugs']:
        print(drug)
        drug = drug.upper()
        temp_prescs = druglist_prescs[druglist_prescs['drug substance name'].str.upper()==drug]
        if(len(temp_prescs))>0:
            drug_amounts = np.array(temp_prescs['amount'])*np.array(temp_prescs['ndd'])
            drug_weights = np.array(temp_prescs['qty'])/np.array(temp_prescs['ndd'])
            pdd = np.average(drug_amounts,weights=drug_weights)
            pdds.loc[len(pdds)]=[drug,druglist['name'], np.round(pdd)]
            assert pd.notnull(pdd)
        else:
            print('temp_prescs not > 0')
        print(pdds)

    #Write PDDs to file for reference
    pdds.to_csv('output/drug_pdds.csv',index=False)

    

In [209]:
# Add condition status (e.g. insomnia count, presence of diabetes, presence of stroke)
for window in sd.exposure_windows:
    print(window['name'],'...')
    pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+ window['name'] +'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)
    #     pt_features = get_multiple_condition_statuses(pt_features,medcoded_entries,window,[codelists.non_stroke_vascular_disease])
    pt_features = create_PDD_columns_for_each_pt(pt_features,window,druglists.all_druglists,prescriptions)
#     pt_features = get_consultation_count(pt_features,all_entries,window)
    pt_features = create_quantiles_and_booleans(pt_features)
    pt_features.to_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+ window['name'] +'.csv',index=False)


12_to_7 ...


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pandas/core/indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


          patid  yob  pracid  female index_date isCase  \
0       8508264   20     264       1 2004-05-27   True   
1       4688370   22     370       1 2007-09-21   True   
2      14416079   16      79       1 2006-08-07   True   
3      11849189   25     189       0 2007-12-19   True   
4       3636234   27     234       0 2007-12-17   True   
5       1165322   20     322       1 2003-10-22   True   
6      11721564   16     564       1 2009-10-26   True   
7       5890114   34     114       0 2008-12-01   True   
8       7329132   23     132       0 2008-10-01   True   
9       4688437   10     437       1 2007-12-19   True   
10      3496322   13     322       1 2003-10-14   True   
11       131310   33     310       0 2005-12-06   True   
12      4917331   31     331       1 2004-06-14   True   
13      3150310   16     310       0 2005-11-02   True   
14     12195076   26      76       1 2007-11-02   True   
15     13087084    9      84       1 2007-12-18   True   
16     1413114

In [210]:
pt_features.columns

Index(['patid', 'yob', 'pracid', 'female', 'index_date', 'isCase',
       'final dementia medcode', 'data_start', 'data_end', 'matchid',
       'age_at_index_date', 'non_insomnia_GP_consultations', 'stroke',
       'non_stroke_vascular_disease', 'hypertension', 'diabetes',
       'mental_illness_non_smi', 'mental_illness_smi', 'sleep_apnoea',
       'chronic_pulmonary_disease', 'epilepsy', 'age_at_index_date:65-69',
       'age_at_index_date:70-74', 'age_at_index_date:75-79',
       'age_at_index_date:80-84', 'age_at_index_date:85-89',
       'age_at_index_date:90-99', 'age_at_index_date:above_99',
       'insomnia_count:0', 'insomnia_count:above_10',
       'non_insomnia_GP_consultations:0', 'insomnia_count:1_5', 'insomnia',
       'insomnia_count:6_10', 'benzo_and_z_drugs_any', 'insomnia_any',
       'non_insomnia_GP_consultations:1_10',
       'non_insomnia_GP_consultations:11_100',
       'non_insomnia_GP_consultations:101_1000',
       'non_insomnia_GP_consultations:above_1000',
 